<a href="https://colab.research.google.com/github/SilvioIA2025/ALura2025/blob/main/Projeto_de_Imers%C3%A3o_IA_Alura_%2B_Google_Gemini.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [229]:
%pip -q install google-genai

In [230]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [233]:
# Configura o cliente da SDK do Gemini

from google import genai
client = genai.Client()
from IPython.display import HTML, Markdown



In [234]:
# Instalar Framework ADK de agentes do Google ################################################
!pip install -q google-adk

In [6]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [235]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [262]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
import textwrap
from rich.markdown import Markdown
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


In [263]:
##################################################################
# --- Agente 1: Buscador de Custos de Viagens Internacionais --- #
##################################################################
def agente_buscador(topico, pais, perfil, data_de_inicio, data_de_fim, numero_adultos, numero_crianças):
    buscador = Agent(
        name="agente_buscador",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
        Como um especialista em calcular gastos em viagens internacionais, meu objetivo é fornecer uma estimativa detalhada e otimizada dos seus custos,
        considerando suas preferências e buscando as melhores opções em cada categoria.

        Instruções para o Especialista em Cálculos de Viagem:

        1. Compreensão do Perfil do Viajante:

        Destino: País(es) e Cidade(s).
        Período: Datas de partida e retorno (para considerar alta/baixa temporada e eventos).
        Número de Viajantes: Adultos e crianças.
        Estilo de Viagem: (e.g., mochileiro, econômico, confortável, luxuoso) para calibrar os valores.
        Interesses Específicos: (e.g., gastronomia, museus, aventura, compras) para alocar orçamento.

        2. Análise e Estimativa por Categoria:

        Voos (Melhores Opções):
        Pesquisar e comparar companhias aéreas, escalas, duração do voo e aeroportos de partida/chegada.
        Considerar voos diretos vs. com escalas (custo/benefício).
        Avaliar épocas de compra (antecedência) e flexibilidade de datas para melhores preços.
        Incluir custos de bagagem despachada, se aplicável.

        Aluguel de Carro:
        Avaliar a necessidade do aluguel versus transporte público no destino.
        Selecionar o tipo de veículo adequado (econômico, SUV, etc.).
        Incluir custos de seguro obrigatório e opcionais, combustível, pedágios e estacionamento.
        Comparar locadoras e políticas de retirada/devolução.

        Hotéis/Hospedagem (Melhores Opções):
        Sugerir tipos de hospedagem (hotéis, hostels, Airbnb, pousadas) com base no estilo de viagem.
        Considerar localização (proximidade a atrações, transporte), avaliações, comodidades (café da manhã, Wi-Fi) e política de cancelamento.
        Estimar custo por noite, ajustado pela temporada.

        Refeições:
        Estimar um custo médio diário por pessoa para café da manhã, almoço e jantar, diferenciando entre restaurantes econômicos, médios e sofisticados.
        Considerar o custo de lanches, bebidas e gorjetas.
        Perguntar se o viajante pretende cozinhar algumas refeições.

        Atrações Turísticas e Atividades:
        Listar as principais atrações pagas e o custo de seus ingressos.
        Sugerir passes turísticos (se vantajosos para a quantidade de atrações).
        Incluir custo de tours guiados, shows ou atividades específicas desejadas.

        Transporte Local:
        Custos de metrô, ônibus, bondes, táxis ou aplicativos de transporte (Uber, Lyft, etc.).
        Estimar passes diários/semanais se forem mais econômicos.
        Despesas Gerais e Extras:

        Seguro Viagem: Item obrigatório e essencial.
        Visto e Taxas de Entrada: Se aplicável ao destino.
        Comunicações: Custo de chip local ou roaming internacional.
        Emergências/Imprevistos: Sugerir uma reserva de 10-15% do total para gastos não planejados.
        Compras/Souvenirs: Definir um orçamento para compras pessoais e lembrancinhas.
        Outros: Lavanderia, gorjetas adicionais, pequenos gastos diversos.

        3. Saída Final:
        Faça voce a escolha de uma cidade a ser visitada no pais, caso o não tenha sido escolhida, retorne todos os resultados de custo pelo
        estilo de Viagem selecionado no perfil (mochileiro, economico, confortavel e luxuoso) e voce deve escolher automaticamente dois
        Interesses Específicos: dentre Gastronomia, Museus, Aventura e Compras.
        Apresentar um orçamento detalhado por categoria (Voos, Aluguel de Carro, Hotéis/Hospedagem, Refeições, Atrações Turísticas e Atividades, Transporte Local, Seguro Viagem
        Visto e Taxas de Entrada, Comunicações, Emergências/Imprevistos, Compras/Souvenirs e Outros ) , com uma estimativa total convertida para a moeda de origem do viajante
        (geralmente BRL) e da moeda local do destino, utilizando a taxa de câmbio atualizada. Oferecer dicas de economia e personalização.

        A sua tarefa é usar a ferramenta de busca do google (google_search) respeitando a data inicial e final da viagem
        Foque em no maximo 4 opções relevantes respeitando os estilos de Viagem: (mochileiro, econômico, confortável, luxuoso), com base no preço, categoria, e avaliaçoes dos clientes anteriores.
        Se uma avaliação tiver poucas informações ou reações entusiasmadas, é possivel que ela não seja tão relevante assim
        e pode ser substituido por outro que tenha mais.
        Essas avaliações relevantes devem ser atuais, de no maximo um tres meses da data de hoje.
        """,
        description = "Agente que busca melhores voos, carros, hoteis, refeiçoes, atrações turisticas, transporte e custos diretos e indiretos de viagem indicado no Google",
        tools=[google_search],
    )
    entrada_do_agente_buscador = f"Tópico: {topico}\nPaís: {pais}\nPerfil: {perfil}\nData de inicio: {data_de_inicio}\nData de fim: {data_de_fim}\nNumero_adultos: {numero_adultos}\nNumero_crianças: {numero_crianças}"
    # Executa o agente
    lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
    return lancamentos_buscados

In [264]:
##########################################
# --- Agente 2: Revisor de Qualidade --- #
##########################################
def agente_revisor(topico, lancamentos_buscados):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
            Você é um Editor e Revisor de Conteúdo meticuloso, especializado em documentos profissionais.
            Por ter um público amplo, entre 18 e 70 anos, use sempre um tom de escrita adequado.
            Revise o lancamentos_buscados abaixo sobre o tópico indicado, verificando clareza, concisão, correção e tom.
            Separar por uma linha em branco cada item da lista.
            Se o lancamentos_buscados estiver bom, repita apenas o texto novamente'.
            Caso haja problemas, corrija e refaça o texto de maneira adequada, gerando uma versão corrigida
            """,
        description="Agente revisor de post para documentos profissionais."
    )
    entrada_do_agente_revisor = f"Tópico: {topico}\nRascunho: {lancamentos_buscados}"
    # Executa o agente
    texto_revisado = call_agent(revisor, entrada_do_agente_revisor)
    return texto_revisado

In [265]:
##################################################################
# --- Agente 3: Buscador da Melhor alternativa de levar ME --- #
##################################################################
def agente_moeda(topico, pais):
    imagem = Agent(
        name="agente_moeda",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
        Voce como especialista em cambio da Viktap, por favor me faça um breve resumo de qual a melhor maneira para eu converter o meu
        dinheiro local para o dinheiro internacional, considerando cartao de credito, cartao de debito, cartao internacional, remessa para conta
        no estrangeiro, ou PIX. No caso do PIX, fale sobre o viktap consultando a fonte no site https://viktap.com.br/
        """,
        tools=[google_search],
    )
    entrada_do_agente_moeda = f"Tópico: {topico}\nPais: {pais}"
    # Executa o agente
    escolha_moeda = call_agent(imagem, entrada_do_agente_moeda)
    return escolha_moeda

In [286]:
####################################################################
# --- Agente 4: Sugestao de Voo, pelo perfil escolhido ----------- #
####################################################################
def agente_viagem(topico, pais, perfil, aeroporto, data_inicio, data_retorno):
    viagem = Agent(
        name="agente_viagem",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
        Como um especialista em viagens internacionais da LATAM, faça apenas uma opção resumida para um voo consultando a LATAM saindo
        do campo aeroporto e na data de ida data_inicio e data de retorno data_retorno
        por favor escolha um voo direto de ida e um voo direto de volta a sua escolha informando o numero do voo e horario de partida e chegada
        Caso a LATAM não possua voo para o destino, faça voce a escolha da companhia aerea, e se tambem nao conseguir voo direto pode escolher com escalas
        """,
        tools=[google_search],
    )
    entrada_do_agente_viagem = f"Tópico: {topico}\nPais: {pais}\nPerfil: {perfil}\nAeroporto: {aeroporto}\nInicio: {data_inicio}\nFim: {data_retorno}"
    # Executa o agente
    viagem_pais = call_agent(viagem, entrada_do_agente_viagem)
    return viagem_pais

In [287]:
################################################################
# --- Agente 5: Sugestao de Hotel, refeição e passeio turistico
#     pelo perfil escolhido ---------------------------------- #
################################################################
def agente_outros(topico, pais, perfil, data_inicio, data_retorno):
    outros = Agent(
        name="agente_outros",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
        Como um especialista em viagens internacionais, faça apenas uma opção resumida para hoteis, refeição e passeios turisticos
        levando em consideração na data de ida data_inicio e data de retorno data_retorno
        por favor escolha somente 3 opções de hoteis, passeios turisticos e refeicoes.
        """,
        tools=[google_search],
    )
    entrada_do_agente_outros = f"Tópico: {topico}\nPais: {pais}\nPerfil: {perfil}\nInicio: {data_inicio}\nFim: {data_retorno}"
    # Executa o agente
    outros = call_agent(outros, entrada_do_agente_outros)
    return outros

In [288]:
######################################
# --- Agente 6: Redator do Post --- #
######################################
def agente_linkedin(topico, texto_revisado):
    linkedin = Agent(
        name="agente_linkedin",
        model="gemini-2.5-flash-preview-04-17",
        instruction="""
            Você é um Redator Criativo especializado em criar posts virais para redes sociais.
            Você escreve posts para a empresa Tervik, a empresa que desenvolveu o pagamento PIX com o Viktap .
            Utilize o tema fornecido no texto_revisado e os pontos mais relevantes fornecidos e, com base nisso,
            escreva um rascunho de post para o Linkedin sobre o tema indicado.
            O post deve ser engajador, informativo, com linguagem simples e incluir 2 a 4 hashtags no final.
            """,
        description="Agente redator de posts engajadores para linkedin"
    )
    entrada_do_agente_linkedin = f"Tópico: {topico}\nPlano de post: {texto_revisado}"
    # Executa o agente
    linkedin = call_agent(linkedin, entrada_do_agente_linkedin)
    return linkedin

In [289]:
##agente_buscador(topico, pais, data_inicio, data_retorno, numero_adultos, numero_crianças)
##agente_revisor(topico, lancamentos_buscados)
##agente_moeda(topico, pais)
##agente_viagem(topico, pais, perfil, aeroporto, data_inicio, data_retorno)
##agente_outros(topico, pais, perfil, data_inicio, data_retorno)
##agente_linkedin(topico, texto_revisado)

from datetime import datetime

def obter_data_do_usuario(param1):
    while True:
        if param1 == 'ida':
          data_str = input("Digite a data de inicio (DD/MM/AAAA): ")
        else:
          data_str = input("Digite a data de retorno (DD/MM/AAAA): ")

        try:
            # Tenta converter a string para um objeto datetime
            data_obj = datetime.strptime(data_str, "%d/%m/%Y")
            return data_obj
        except ValueError:
            print("Formato de data inválido. Por favor, use DD/MM/AAAA.")

def selecionar_perfil(opcoes, mensagem="Selecione uma opção:"):
    print(mensagem)
    for i, opcao in enumerate(opcoes):
        print(f"[{i + 1}] {opcao}")

    while True:
        try:
            escolha = input("Digite o número da sua escolha: ")
            indice_escolhido = int(escolha) - 1 # Ajusta para o índice da lista (0-based)

            if 0 <= indice_escolhido < len(opcoes):
                return opcoes[indice_escolhido]
            else:
                print("Número inválido. Por favor, digite um número da lista.")
        except ValueError:
            print("Entrada inválida. Por favor, digite um número.")



In [290]:
topico = 'Viagem'
pais = input("Digite o Pais/Cidade do destino: ")
data_inicio = obter_data_do_usuario('ida')
data_retorno = obter_data_do_usuario('fim')
adultos = input("Digite o numero de adultos: ")
criancas = input("Digite o numero de crianças: ")
opcoes = ["Mochileiro", "Economico", "Confortavel", "Luxuoso"]
perfil = selecionar_perfil(opcoes, mensagem="Selecione uma opção:")
print (perfil)
aeroporto = input("Digite o aeroporto de partida: ")

lancamentos_buscados = agente_buscador(topico, pais, perfil, data_inicio, data_retorno, adultos, criancas)
##print("\n Resultado do Agente 1 (Buscador)! \n")
##display(to_markdown(lancamentos_buscados))
##print("----------------------------------------------------------------")

post_final = agente_revisor(topico, lancamentos_buscados)
display(to_markdown("-------------------------------"))
display(to_markdown("Recomendações para a sua viagem"))
display(to_markdown("-------------------------------"))
display(to_markdown(post_final))

escolha_moeda = agente_moeda(topico, pais)
display(to_markdown("--------------------------------------------------"))
display(to_markdown("Dicas financeiras para pagar com moeda estrangeira"))
display(to_markdown("--------------------------------------------------"))
display(to_markdown(escolha_moeda))

viagem_pais = agente_viagem(topico, pais, perfil, aeroporto, data_inicio, data_retorno)
display(to_markdown("------------------------------------"))
display(to_markdown("Dica de um agente de viagem da LATAM"))
display(to_markdown("------------------------------------"))
display(to_markdown(viagem_pais))

outros = agente_outros(topico, pais, perfil, data_inicio, data_retorno)
display(to_markdown("---------------------------------------------------------"))
display(to_markdown("Algumas dicas de Hoteis, refeiçoes e passeios turisticos"))
display(to_markdown("---------------------------------------------------------"))
display(to_markdown(outros))

linkedin = agente_linkedin(topico, post_final)
display(to_markdown("---------------------------------------------------------"))
display(to_markdown("Post do LinkedIn"))
display(to_markdown("---------------------------------------------------------"))
display(to_markdown(linkedin))


Digite o Pais/Cidade do destino: Amsterdan
Digite a data de inicio (DD/MM/AAAA): 15/06/2025
Digite a data de retorno (DD/MM/AAAA): 30/06/2025
Digite o numero de adultos: 1
Digite o numero de crianças: 0
Selecione uma opção:
[1] Mochileiro
[2] Economico
[3] Confortavel
[4] Luxuoso
Digite o número da sua escolha: 2
Economico
Digite o aeroporto de partida: GRU


▌ ───────────────────────────────────────────────────────────────────────────────────────────────────────────────

▌ Recomendações para a sua viagem                                                                                

▌ ───────────────────────────────────────────────────────────────────────────────────────────────────────────────

▌ Olá! Como seu especialista em gastos de viagem internacional, vou te ajudar a planejar e estimar os custos para
▌ sua aventura econômica em Amsterdã, Holanda, de 15 a 30 de junho de 2025.                                      
▌ Considerando seu perfil "Econômico" e a cidade de Amsterdã, focaremos em otimizar os gastos sem deixar de      
▌ aproveitar a cultura e a atmosfera local. Escolhi como seus interesses específicos Museus e Gastronomia        
▌ (explorando opções acessíveis e locais).                                                                       
▌ Vamos detalhar as estimativas por categoria:                                                                   
▌ Taxa de Câmbio de Referência:                                                                                  
▌  • Euro (€) para Real (BRL): 1 EUR = Aproximadamente 6.32 BRL (em [Data Atual de Consulta]). Note: A taxa de   
▌    câmbio é apenas uma referência e pode variar significativamente até a data da viagem.                       
▌ Estimativa Detalhada por Categoria:                                                                            
▌ Para um perfil econômico em Amsterdã, por 15 dias (14 noites):                                                 
▌  • Voos (Ida e Volta - Econômica): Pesquisas indicam que passagens aéreas do Brasil para a Holanda em junho de 
▌    2025 podem variar. Encontramos ofertas a partir de BRL 4.154. Considerando um valor médio para um voo       
▌    econômico com pelo menos uma escala, que geralmente oferece melhor custo-benefício, estimamos:              
▌     • Estimativa: € 700 - € 900 (aproximadamente BRL 4.424 - BRL 5.688)                                        
▌  • Aluguel de Carro: Em Amsterdã, o transporte público é eficiente e a cidade é plana, ideal para ser explorada
▌    a pé ou de bicicleta. Alugar um carro geralmente não é necessário e pode ser caro (combustível,             
▌    estacionamento, pedágios). Para um perfil econômico, não incluiremos aluguel de carro.                      
▌     • Estimativa: € 0                                                                                          
▌  • Hotéis/Hospedagem (Econômica): Focaremos em opções como hostels, hotéis mais simples em bairros centrais ou 
▌    bem conectados por transporte público. Junho é alta temporada.                                              
▌     • Hostels ou hotéis econômicos: diárias a partir de € 32 por pessoa em hostels. Hotéis baratos com boas    
▌       avaliações podem ter diárias a partir de R$ 445 (aproximadamente € 70) para 2 pessoas, mas para 1 pessoa 
▌       em quarto privativo, espere um valor um pouco maior.                                                     
▌     • Estimativa (14 noites): € 500 - € 800 (aproximadamente BRL 3.160 - BRL 5.056) - considerando uma média de
▌       € 35 a € 57 por noite em hostels ou hotéis econômicos.                                                   
▌  • Refeições (Econômica): Combinaremos refeições em restaurantes acessíveis, mercados e talvez cozinhar algo   
▌    simples.                                                                                                    
▌     • Refeição em restaurante econômico: a partir de € 15 por pessoa. Lanches rápidos ou comida de rua: € 5 a €
▌       10. Café da manhã: € 5 a € 15.                                                                           
▌     • Considerando um gasto médio diário focado em economia: € 30 - € 40 por dia.                              
▌     • Estimativa (15 dias): € 450 - € 600 (aproximadamente BRL 2.844 - BRL 3.792)                              
▌  • Atrações Turísticas e Atividades (Foco em Museus): Amsterdã tem muitos museus. Para um perfil econômico,    
▌    selecionaremos os imperdíveis e buscaremos por passes ou descontos.                                         
▌     • Ingresso Museu Van Gogh: a partir de € 22.                                                               
▌     • In

...

▌ Com base no seu perfil econômico e nas datas de 15 a 30 de junho de 2025, preparei algumas opções resumidas    
▌ para sua viagem a Amsterdã:                                                                                    
▌ Hotéis Econômicos:                                                                                             
▌  1 Budget Hotel Tourist Inn: Bem avaliado e próximo à Estação Central de Amsterdã.                             
▌  2 Bunk Hotel Amsterdam: Com boas avaliações e a cerca de 2.4 km do centro da cidade.                          
▌  3 easyHotel Amsterdam City Centre South: Localizado no distrito de De Pijp, com boa relação custo-benefício e 
▌    acesso a atrações a pé ou por transporte público.                                                           
▌ Refeições Econômicas:                                                                                          
▌  1 Mercados locais e supermercados: Explorar mercados como o Albert Cuyp Market ou simplesmente comprar lanches
▌    e ingredientes em supermercados como Albert Heijn, Jumbo, Lidl, Vomar ou Spar pode ser uma opção muito      
▌    econômica.                                                                                                  
▌  2 Comida de rua: Experimente opções como Leeman Döner (kebabs) ou Roopram Roti (comida surinamesa) para       
▌    refeições saborosas e baratas. Snacks tradicionais holandeses como bitterballen também podem ser encontrados
▌    em bares e lanchonetes.                                                                                     
▌  3 Cafés e eetcafe: Locais como Cultureel Eetcafe Skek, administrado por estudantes, oferecem pratos caseiros a
▌    preços acessíveis. Alguns cafés também podem ter opções de almoço a preços razoáveis.                       
▌ Passeios Turísticos (Foco em opções econômicas e gratuitas):                                                   
▌  1 Caminhar e explorar os canais: Passear pela área dos canais, Patrimônio Mundial da UNESCO, é uma atividade  
▌    gratuita e imperdível para apreciar a arquitetura e a atmosfera da cidade. Considere explorar áreas como    
▌    Herengracht, Keizersgracht, Prinsengracht e Brouwersgracht.                                                 
▌  2 Visitar parques e jardins: Aproveite os espaços verdes de Amsterdã, como o Vondelpark, Westerpark e os      
▌    jardins do Rijksmuseum (gratuitos para visitar). Em junho, o Vondelpark costuma ter apresentações gratuitas 
▌    em seu teatro ao ar livre.                                                                                  
▌  3 Descobrir arte de rua e mercados: Explore a vibrante cena de arte de rua em locais como NDSM Wharf e De     
▌    Pijp. Visite mercados como o Bloemenmarkt (mercado de flores flutuante) ou o Waterlooplein Flea Market para 
▌    uma experiência local.                                                                                      

▌ ───────────────────────────────────────────────────────────────────────────────────────────────────────────────

▌ Post do LinkedIn                                                                                               

▌ ───────────────────────────────────────────────────────────────────────────────────────────────────────────────

▌ Okay, aqui está um rascunho de post para o LinkedIn, utilizando as informações fornecidas sobre o orçamento de 
▌ viagem econômica para Amsterdã e integrando a Tervik/Viktap como facilitador financeiro.                       
▌ ───────────────────────────────────────────────────────────────────────────────────────────────────────────────
▌ Rascunho de Post para LinkedIn - Tervik/Viktap                                                                 
▌ Hook: Sonha em conhecer Amsterdã sem estourar o orçamento? 🇳🇱                                                  
▌ Corpo:                                                                                                         
▌ Planejar uma viagem internacional exige cuidado, especialmente quando o objetivo é economizar sem perder as    
▌ melhores experiências.                                                                                         
▌ Pensando nisso, simulamos um orçamento detalhado para uma aventura de 15 dias em Amsterdã com foco em          
▌ experiências (museus incríveis e gastronomia local acessível) e muita inteligência financeira! 💡              
▌ Com um planejamento inteligente, estimamos que é possível viver essa experiência incrível com um investimento  
▌ total entre R$ 13.854 e R$ 19.698 (considerando a taxa de câmbio atual para o Euro). 📊                        
▌ As chaves para o sucesso? Pesquisa de voos e hospedagem com antecedência, uso eficiente do transporte público  
▌ (ou bike!), explorar mercados e restaurantes locais fora das áreas turísticas. Pequenas decisões que fazem     
▌ grande diferença no bolso! 💰                                                                                  
▌ E na hora de gerenciar seus gastos durante a viagem? Fazer câmbio, lidar com taxas, pagamentos... Pode parecer 
▌ complicado, mas com a tecnologia certa, não precisa ser! ✅                                                    
▌ É aqui que Tervik e o poder do PIX via Viktap entram para simplificar sua vida, oferecendo uma solução moderna 
▌ e eficiente para você focar no que realmente importa: VIVER a viagem sem preocupações financeiras              
▌ desnecessárias! ✨                                                                                             
▌ Pronto(a) para tirar o sonho de Amsterdã do papel? Comece pelo planejamento financeiro! 😉                     
▌ Qual sua maior dica ao planejar o orçamento de uma viagem internacional? Compartilhe nos comentários!          
▌ ───────────────────────────────────────────────────────────────────────────────────────────────────────────────
▌ Hashtags:                                                                                                      
▌ #ViagemInternacional #PlanejamentoFinanceiro #Amsterda #Tervik                                                 